# Mining High Voltage - Grafana Analysis
Extract data from all MiningHighVoltage CSV files in sample_outputs and prepare for Grafana visualization

In [3]:
import pandas as pd
from pathlib import Path
import json
import warnings
warnings.filterwarnings('ignore')

# ===== SETUP =====
sample_outputs_dir = Path("sample_outputs")

# Find all MiningHighVoltage files
files = sorted([f for f in sample_outputs_dir.glob("*MiningHighVoltage*.csv")])

print(f"Found {len(files)} MiningHighVoltage files")

# ===== LOAD DATA =====
all_data_complete = []
for file_path in files:
    try:
        df = pd.read_csv(file_path)
        all_data_complete.append(df)
    except:
        pass

print(f"Successfully loaded {len(all_data_complete)} files")

if len(all_data_complete) == 0:
    print("Could not load any files!")
else:
    # ===== COMBINE DATA =====
    combined_df = pd.concat(all_data_complete, ignore_index=True)

    print(f"\n{'='*60}")
    print(f"COMPLETE DATASET STATISTICS")
    print(f"{'='*60}")
    print(f"Total files analyzed: {len(files)}")
    print(f"Total data points: {len(combined_df):,}")
    print(f"Energy range: {combined_df['Energy (keV)'].min():.2f} to {combined_df['Energy (keV)'].max():.2f} keV")
    print(f"Intensity range: {combined_df['Intensity (CPS)'].min():.2f} to {combined_df['Intensity (CPS)'].max():.2f} CPS")
    
    # ===== EXPORT DATA FOR GRAFANA =====
    print(f"\n{'='*60}")
    print("Exporting data for Grafana...")
    print(f"{'='*60}")
    
    # Export as CSV
    csv_file = "mining_high_voltage_data.csv"
    combined_df.to_csv(csv_file, index=False)
    print(f"✓ CSV exported: {csv_file}")
    
    # Export as JSON for API
    json_file = "mining_high_voltage_data.json"
    combined_df.to_json(json_file, orient='records', indent=2)
    print(f"✓ JSON exported: {json_file}")
    
    # Create summary statistics JSON
    stats_json = {
        "total_files": len(files),
        "total_data_points": len(combined_df),
        "energy_min": float(combined_df['Energy (keV)'].min()),
        "energy_max": float(combined_df['Energy (keV)'].max()),
        "energy_mean": float(combined_df['Energy (keV)'].mean()),
        "intensity_min": float(combined_df['Intensity (CPS)'].min()),
        "intensity_max": float(combined_df['Intensity (CPS)'].max()),
        "intensity_mean": float(combined_df['Intensity (CPS)'].mean()),
        "non_zero_points": int((combined_df['Intensity (CPS)'] > 0).sum()),
    }
    
    with open("mining_high_voltage_stats.json", "w") as f:
        json.dump(stats_json, f, indent=2)
    print(f"✓ Statistics JSON exported: mining_high_voltage_stats.json")
    
    print(f"\n{'='*60}")
    print("NEXT STEPS FOR GRAFANA")
    print(f"{'='*60}")
    print("\n1. Install Docker: https://www.docker.com/products/docker-desktop")
    print("\n2. Run Grafana in Docker:")
    print("   docker run -d -p 3000:3000 --name grafana grafana/grafana")
    print("\n3. Access Grafana at: http://localhost:3000")
    print("   Login: admin / admin")
    print("\n4. Add Data Source:")
    print("   - Settings → Data Sources → Add Data Source")
    print("   - Choose 'JSON API'")
    print("   - URL: http://host.docker.internal:5000/api/data")
    print("\n5. Create Dashboard:")
    print("   - New Dashboard")
    print("   - Add Panel with JSON API as datasource")
    print("   - Plot 'Energy (keV)' vs 'Intensity (CPS)'")
    print("\n6. Run Python API Server (next cell):")
    print("   - Execute the next cell to start the JSON API server")
    print("   - Server runs on http://localhost:5000")
    
    print(f"\n{'='*60}")
    print("✓ Data Export Complete!")
    print(f"{'='*60}")


Found 150 MiningHighVoltage files
Successfully loaded 150 files

COMPLETE DATASET STATISTICS
Total files analyzed: 150
Total data points: 307,200
Energy range: -11.06 to 41012.13 keV
Intensity range: 0.00 to 521.79 CPS

Exporting data for Grafana...
✓ CSV exported: mining_high_voltage_data.csv
✓ JSON exported: mining_high_voltage_data.json
✓ Statistics JSON exported: mining_high_voltage_stats.json

NEXT STEPS FOR GRAFANA

1. Install Docker: https://www.docker.com/products/docker-desktop

2. Run Grafana in Docker:
   docker run -d -p 3000:3000 --name grafana grafana/grafana

3. Access Grafana at: http://localhost:3000
   Login: admin / admin

4. Add Data Source:
   - Settings → Data Sources → Add Data Source
   - Choose 'JSON API'
   - URL: http://host.docker.internal:5000/api/data

5. Create Dashboard:
   - New Dashboard
   - Add Panel with JSON API as datasource
   - Plot 'Energy (keV)' vs 'Intensity (CPS)'

6. Run Python API Server (next cell):
   - Execute the next cell to start the

In [ ]:
# Optional: Start Flask API Server for Grafana (run in background)
# Install Flask first if needed: pip install flask flask-cors

try:
    from flask import Flask, jsonify
    from flask_cors import CORS
    import json
    from threading import Thread
    
    app = Flask(__name__)
    CORS(app)
    
    # Load the data
    with open("mining_high_voltage_data.json", "r") as f:
        data = json.load(f)
    
    with open("mining_high_voltage_stats.json", "r") as f:
        stats = json.load(f)
    
    @app.route('/api/data', methods=['GET'])
    def get_data():
        """Return mining data for Grafana"""
        return jsonify(data)
    
    @app.route('/api/stats', methods=['GET'])
    def get_stats():
        """Return statistics"""
        return jsonify(stats)
    
    @app.route('/health', methods=['GET'])
    def health():
        """Health check endpoint"""
        return jsonify({"status": "ok"})
    
    def run_server():
        app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)
    
    # Start server in background thread
    server_thread = Thread(target=run_server, daemon=True)
    server_thread.start()
    
    print("✓ Flask API Server started on http://localhost:5000")
    print("  - /api/data → Get all mining data")
    print("  - /api/stats → Get statistics")
    print("  - /health → Health check")
    print("\nGrafana can now connect to: http://localhost:5000/api/data")
    
except ImportError:
    print("Flask not installed. Install with: pip install flask flask-cors")
    print("Then run this cell again")


✓ Flask API Server started on http://localhost:5000
  - /api/data → Get all mining data
  - /api/stats → Get statistics
  - /health → Health check

Grafana can now connect to: http://localhost:5000/api/data


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.20.211.0:5000
Press CTRL+C to quit
